In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,mls,mls
band,[10],8
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,0
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,"[0, -1, 0]"
ng_refs,-,"[3, 2, 3]"
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.134107                    1 -0.162552
0.000750        2 -0.133502                    2 -0.161727
0.001052        3 -0.130775                    3 -0.159768
0.001476        4 -0.127989                    4 -0.157601
0.002070        5 -0.125396                    5 -0.155571
0.002904        6 -0.123131                    6 -0.153516
0.004074        7 -0.121313                    7 -0.151726
0.005714        8 -0.120101                    8 -0.150040
0.008015        9 -0.119749                    9 -0.148853
0.011243       10 -0.120530                   10 -0.148045
0.015771       11 -0.122859                   11 -0.148064
0.022122       12 -0.127290                   12 -0.149318
0.031031       13 -0.134674                   13 -0.152183
0.043528       14 -0.146204                   14 -0.157427
0.061057       15 -0.154468                   15 -0.159198
0.085645       16 -0.141828                   16 -0.144358
0.120136       17 -0.107541                   17 -0.110946
0.168516       18 -0.065559                   18 -0.067622
0.236378       19 -0.016042                   19 -0.015349
0.331549       20  0.043121                   20  0.046725
0.465100       21  0.116778                   21  0.122279
0.652400       22  0.220287                   22  0.222228
0.915100       23  0.326819                   23  0.318661
1.283650       24  0.292899                   24  0.282389
1.800600       25  0.169545                   25  0.163670
2.525700       26  0.093714                   26  0.095604
3.542800       27  0.056653                   27  0.061806
4.969550       28  0.038365                   28  0.040818
6.970850       29  0.029634                   29  0.029658
9.778100       30  0.024466                   30  0.024610
13.715850      31  0.019408                   31  0.021051
19.239350      32  0.014145                   32  0.016268
26.987250      33  0.010873                   33  0.012683
37.855300      34  0.009286                   34  0.011107
53.100050      35  0.006934                   35  0.008260
73.887500      36  0.003756                   36  0.003445
97.662500      37  0.001862                   37  0.000738
121.437500     38  0.001142                   38  0.000353
145.212500     39 -0.000864                   39 -0.001192
168.987500     40 -0.003060                   40 -0.002978
192.762500     41 -0.000992                   41  0.000002
216.537500     42  0.014138                   42  0.019887
240.312500     43  0.039087                   43  0.050387
264.087500     44  0.067254                   44  0.058275
287.862500     45  0.092458                   45  0.063163
311.637500     46  0.099109                   46  0.083940
335.412500     47  0.105350                   47  0.104973
359.187500     48  0.114194                   48  0.115970
382.962500     49  0.122338                   49  0.120200
406.737500     50  0.128138                   50  0.123906
430.512500     51  0.130195                   51  0.130433
454.287500     52  0.126481                   52  0.133408
478.062500     53  0.120707                   53  0.136371
501.837500     54  0.114206                   54  0.132461
525.612500     55  0.107169                   55  0.123679
549.387500     56  0.100231                   56  0.110767
573.162500     57  0.094581                   57  0.097775
596.937500     58  0.091548                   58  0.088400
620.712500     59  0.083912                   59  0.076587
644.487500     60  0.073717                   60  0.065357
668.262500     61  0.063024                   61  0.056945
692.037500     62  0.052869                   62  0.045826
715.812500     63  0.044022                   63  0.041194
739.587500     64  0.037069                   64  0.035825
763.362500     65  0.032806                   65  0.028783
78

# Fluxes by Layer

CRD                          CLIRAD                \
                        flug       fldg     fnetg       flug          fldg   
pressure    level                                                            
0.000000    1      -7.292049   0.000000 -7.292049  -7.160299 -1.160315e-08   
0.000624    2      -7.292064   0.000005 -7.292059  -7.160314  3.424641e-06   
0.000876    3      -7.292070   0.000007 -7.292063  -7.160320  4.825715e-06   
0.001229    4      -7.292078   0.000010 -7.292068  -7.160329  6.840902e-06   
0.001723    5      -7.292090   0.000015 -7.292075  -7.160341  9.746762e-06   
0.002417    6      -7.292107   0.000021 -7.292086  -7.160358  1.392988e-05   
0.003391    7      -7.292130   0.000030 -7.292100  -7.160382  1.995474e-05   
0.004757    8      -7.292162   0.000042 -7.292120  -7.160415  2.861476e-05   
0.006672    9      -7.292207   0.000060 -7.292147  -7.160462  4.106167e-05   
0.009359    10     -7.292269   0.000084 -7.292185  -7.160527  5.889906e-05   
0.013128    11     -7.292358   0.000119 -7.292239  -7.160618  8.443711e-05   
0.018415    12     -7.292483   0.000167 -7.292316  -7.160748  1.208703e-04   
0.025830    13     -7.292660   0.000232 -7.292428  -7.160930  1.725917e-04   
0.036232    14     -7.292915   0.000321 -7.292593  -7.161191  2.456284e-04   
0.050823    15     -7.293286   0.000440 -7.292846  -7.161566  3.480220e-04   
0.071291    16     -7.293829   0.000608 -7.293221  -7.162103  4.991978e-04   
0.100000    17     -7.294596   0.000893 -7.293703  -7.162852  7.569823e-04   
0.140271    18     -7.295633   0.001417 -7.294216  -7.163857  1.233170e-03   
0.196760    19     -7.296999   0.002345 -7.294655  -7.165171  2.094552e-03   
0.275997    20     -7.298732   0.003927 -7.294805  -7.166822  3.601915e-03   
0.387100    21     -7.300786   0.006548 -7.294238  -7.168753  6.147084e-03   
0.543100    22     -7.302868   0.010789 -7.292080  -7.170643  1.029741e-02   
0.761700    23     -7.303848   0.017472 -7.286376  -7.171393  1.680289e-02   
1.068500    24     -7.301339   0.026841 -7.274498  -7.168858  2.584936e-02   
1.498800    25     -7.295243   0.035675 -7.259568  -7.162938  3.432536e-02   
2.102400    26     -7.288156   0.040710 -7.247446  -7.156037  3.912760e-02   
2.949000    27     -7.281011   0.042963 -7.238048  -7.149055  4.173489e-02   
4.136600    28     -7.274021   0.043943 -7.230078  -7.142205  4.358059e-02   
5.802500    29     -7.267257   0.044750 -7.222507  -7.135545  4.497626e-02   
8.139200    30     -7.260546   0.046242 -7.214304  -7.128839  4.648057e-02   
11.417000   31     -7.253652   0.048847 -7.204805  -7.121811  4.900846e-02   
16.014700   32     -7.246767   0.052532 -7.194234  -7.114599  5.326327e-02   
22.464000   33     -7.240745   0.057317 -7.183428  -7.108062  5.915573e-02   
31.510500   34     -7.236023   0.064247 -7.171776  -7.102705  6.739156e-02   
44.200100   35     -7.232438   0.074621 -7.157817  -7.098473  7.985631e-02   
62.000000   36     -7.231724   0.088529 -7.143195  -7.097282  9.608453e-02   
85.775000   37     -7.238364   0.105746 -7.132618  -7.104244  1.127507e-01   
109.550000  38     -7.251139   0.123764 -7.127375  -7.117898  1.284817e-01   
133.325000  39     -7.268015   0.143857 -7.124158  -7.135789  1.473665e-01   
157.100000  40     -7.291051   0.164459 -7.126593  -7.160278  1.684983e-01   
180.875000  41     -7.323782   0.188573 -7.135209  -7.195271  1.951025e-01   
204.650000  42     -7.376308   0.238306 -7.138002  -7.251703  2.515413e-01   
228.425000  43     -7.479923   0.381737 -7.098186  -7.360651  4.165029e-01   
252.200000  44     -7.651092   0.662988 -6.988104  -7.539392  7.371655e-01   
275.975000  45     -7.907381   1.108686 -6.798696  -7.806813  1.168725e+00   
299.750000  46     -8.268424   1.730118 -6.538306  -8.175794  1.715610e+00   
323.525000  47     -8.702170   2.442987 -6.259183  -8.614991  2.391234e+00   
347.300000  48     -9.193512   3.231027 -5.962485  -9.109158  3.181069e+00   
371.075000  49     -9.739708   4.098830 -5.6408

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')